California Housing Dataset

- MedInc — медианный уровень дохода в квартале;
- HouseAge — медианный возраст дома в квартале;
- AveRooms — среднее количество помещений;
- AveBedrms — среднее количество спальных комнат;
- Population — население квартала;
- AveOccup — средний срок проживания;
- Latitude — значение широты квартала;
- Longitude — значение долготы квартала;
- Price — целевое значение.

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

data = fetch_california_housing()

In [2]:
df = pd.DataFrame(data["data"],columns=data["feature_names"])
df.loc[:,"target"] = data["target"]

df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [3]:
def rmse(y_hat,y):
    return mean_squared_error(y_hat,y,squared=False)

X = df.drop("target",axis=1)
y = df['target']
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=42)

print(f'Размер обучающей выборки {X_train.shape}')
print(f'Размер тестовой выборки {X_test.shape}')

Размер обучающей выборки (15480, 8)
Размер тестовой выборки (5160, 8)


Обучим простейший пайплайн, вызвав метод fit(), состоящий из случайного леса и стандартизации (StandardScaler). На вход пайплайна подается список из преобразований в формате кортежа (название метода преобразования, по которому мы будем обращаться в дальнейшем к нашему преобразованию, объект  метода преобразования). Например:

In [4]:
pipeline = Pipeline([('scaler', StandardScaler()), ('rf', RandomForestRegressor())])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

In [5]:
y_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по метрике R2: 0.8093
Качество по RSME: 0.5023


In [6]:
pipeline.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()), ('rf', RandomForestRegressor())],
 'verbose': False,
 'scaler': StandardScaler(),
 'rf': RandomForestRegressor(),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'rf__bootstrap': True,
 'rf__ccp_alpha': 0.0,
 'rf__criterion': 'squared_error',
 'rf__max_depth': None,
 'rf__max_features': 1.0,
 'rf__max_leaf_nodes': None,
 'rf__max_samples': None,
 'rf__min_impurity_decrease': 0.0,
 'rf__min_samples_leaf': 1,
 'rf__min_samples_split': 2,
 'rf__min_weight_fraction_leaf': 0.0,
 'rf__n_estimators': 100,
 'rf__n_jobs': None,
 'rf__oob_score': False,
 'rf__random_state': None,
 'rf__verbose': 0,
 'rf__warm_start': False}

In [7]:
print(pipeline[1].n_estimators)
print(pipeline['rf'].n_estimators)

100
100


In [8]:
pipeline.set_params(rf__n_estimators=200)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf', RandomForestRegressor(n_estimators=200))])

In [9]:
from sklearn.model_selection import GridSearchCV
param_grid = {'scaler__with_mean':[True,False],
              'rf__n_estimators':[100, 200, 500]}
grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose = True)

In [10]:
"""%time grid_search.fit(X_train,y_train)
print(grid_search.best_estimator_)"""

'%time grid_search.fit(X_train,y_train)\nprint(grid_search.best_estimator_)'

In [11]:
"""y_pred = grid_search.best_estimator_.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')"""

"y_pred = grid_search.best_estimator_.predict(X_test)\nprint(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')\nprint(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')"

## <center> Предобработка в пайплайнах

In [12]:
df_wine = pd.read_csv("data/Red.csv")
df_wine.head(5)

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016


In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

ct = make_column_transformer(
    (StandardScaler(),["Price"]),
    (OneHotEncoder(),["Country"])
)
print(ct)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])


In [14]:
pipeline = Pipeline([("ct", ct),("rf", RandomForestRegressor())])

In [15]:
X = df_wine[["Country", "Price"]]
y = df_wine["Rating"]

In [16]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor())])

In [17]:
pd.DataFrame(
    pipeline['ct'].transform(X).toarray(),
    columns=["price"] + pipeline["ct"].transformers_[1][1].get_feature_names_out().tolist()
)

,price,Country_Argentina,Country_Australia,Country_Austria,Country_Brazil,Country_Bulgaria,Country_Canada,Country_Chile,Country_China,Country_Croatia,...,Country_Portugal,Country_Romania,Country_Slovakia,Country_Slovenia,Country_South Africa,Country_Spain,Country_Switzerland,Country_Turkey,Country_United States,Country_Uruguay
0,0.657648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.278402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.373184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.358231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.117684,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8661,-0.266981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8662,-0.224358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8663,-0.178910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8664,-0.387784,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
import joblib
joblib.dump(pipeline,"data/pipeline.pkl")

['data/pipeline.pkl']

In [19]:
pipeline_loaded = joblib.load('data/pipeline.pkl')

In [20]:
print(pipeline_loaded)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor())])


Задание 6.1

На основании датасета предсказания рейтинга красного вина, с которым вы работали в разделе о предобработке данных в предыдущем юните (файл Red.csv ), вам предстоит выполнить следующее:

Добавить в пайплайн, созданный вами в предыдущем юните, обработку столбца 'Region'. Кодирование 'Region' произвести с использованием OrdinalEncoder. Стандартизацию столбца 'Price' и кодирование столбца 'Country' осуществить в соответствии с примером, представленным в юните 5.
Важно! Для совпадения результатов процесс трансформации столбцов должен выполняться в следующей последовательности:

- Кодирование столбца 'Region' через OrdinalEncoder.
- Стандартизация столбца 'Price'.
- Кодирование столбца 'Country' через OneHotEncoder.

Обучить пайплайн на тренировочном наборе данных (файл Red.csv ) и оценить качество модели по метрике RMSE на тестовом наборе (файл Red_test.csv ).
Зафиксировать random_state=42.
Сохранить пайплайн в файл pipeline_wine.pkl.
В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [21]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer

ct = make_column_transformer(
    (OrdinalEncoder(),["Region"]),
    (StandardScaler(),["Price"]),
    (OneHotEncoder(),["Country"])
)
print(ct)

ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                 ['Region']),
                                ('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])


In [22]:
pipeline = Pipeline([("ct", ct),("rf", RandomForestRegressor(random_state=42))])

In [23]:
X = df_wine[["Country", "Price","Region"]]
y = df_wine["Rating"]

In [24]:
red_test = pd.read_csv("data/Red_test.csv")
red_test.head(5)

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
1,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
2,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
3,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
4,Capatosta 2015,Italy,Toscana,Poggio Argentiera,3.8,101,19.90,2015


In [25]:
X_test = red_test[["Country", "Price","Region"]]
y_test = red_test["Rating"]

In [26]:
pipeline.fit(X, y)
y_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по метрике R2: 0.9393
Качество по RSME: 0.0765


Задание 6.2
1 point possible (graded)
Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр n_estimators в случайном лесу со значения по умолчанию до 200 , используя метод set_params.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [27]:
pipeline.set_params(rf__n_estimators=200)
pipeline.fit(X, y)

y_pred = pipeline.predict(X_test)

print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по RSME: 0.0761


6.3 

Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:

Собрать StackingRegressor:

- В качестве базовых моделей возьмите ридж-регрессию RidgeCV() и решающее дерево.
- В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
- Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме random_state).
- Зафиксировать random_state=42 (для всех моделей).

Заменить в пайплайне задачи 6.1 случайный лес на StackingRegressor.
Обучить модель на тренировочной выборке.
В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округлённый до второго знака после запятой.

In [28]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor

estimators = [
    ("lr",RidgeCV()),
    ("dt", DecisionTreeRegressor(random_state=42))
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)

ct = make_column_transformer(
    (OrdinalEncoder(),["Region"]),
    (StandardScaler(),["Price"]),
    (OneHotEncoder(),["Country"])
)

In [29]:
pipeline = Pipeline([("ct", ct),("rf", reg)])

In [30]:
X = df_wine[["Country", "Price","Region"]]
y = df_wine["Rating"]

In [31]:
X_test = red_test[["Country", "Price","Region"]]
y_test = red_test["Rating"]

In [32]:
pipeline.fit(X, y)
y_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по метрике R2: 0.6509
Качество по RSME: 0.1834


In [35]:
estimators = [
    ('lr', RidgeCV()),
    ('dt',  DecisionTreeRegressor(random_state=42))
]
 
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)
 
 
from sklearn.preprocessing import OrdinalEncoder
df_wine= pd.read_csv('data/Red.csv')
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']
 
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
     (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']),
    )
 
pipeline = Pipeline([('ct', ct), ('stack',reg) ])
pipeline.fit(X,y)
 
 
df_wine_test = pd.read_csv('data/Red_test.csv')
X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по RSME: 0.1834
